In [1]:
from os import listdir
from os.path import isfile, join
import os
import re
import pandas
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
onlyfiles = [f for f in listdir(os.getcwd()) if isfile(join(os.getcwd(), f))]

In [2]:
def reader_fos(fileName):
    df = pandas.read_json(fileName, lines=True)
    fos_authors_and_year = df[["id", "title", "year", "authors", "fos", "references"]]
    return fos_authors_and_year

In [3]:
dblp_part_regex = re.compile("author_indexed_fos*")
part_files_filter_obj = filter(lambda file_name: dblp_part_regex.match(file_name), onlyfiles)
part_files = list(part_files_filter_obj)

In [4]:
list_of_dataframes = list(map(reader_fos, part_files))
df = pandas.concat(list_of_dataframes, ignore_index=True)

In [65]:
filtered_years = df.loc[df["year"] > 2010]
filtered_years = filtered_years.loc[df["year"] < 2018]
filtered_years

,id,title,year,authors,fos,references
5,1965664720,A low power wireless sensor node with vibratio...,2014.0,"[{'index': 2247601}, {'index': 1264008}, {'ind...","[{'name': '104', 'w': 0.0}, {'name': '649', 'w...","[1510600213, 2170239483]"
6,1965664774,Shortest Paths Based Web Service Selection in ...,2014.0,"[{'index': 2247603}, {'index': 2247604}]","[{'name': '2716', 'w': 0.5306294}, {'name': '2...","[1479916289, 1560267311, 2002838096, 200757299..."
12,1965665477,Contracts for First-Class Classes,2013.0,"[{'index': 465043}, {'index': 457705}, {'index...","[{'name': '10', 'w': 0.4498527}, {'name': '227...","[204817905, 1518650316, 1526601993, 1601712140..."
16,1965665804,An interpretation of the Cover and Leung capac...,2012.0,"[{'index': 283325}, {'index': 580777}]","[{'name': '0', 'w': 0.43447753800000005}, {'na...","[1489262227, 2011914761, 2014996477, 201797019..."
20,1965666279,Implementing Linguistic Landscape investigatio...,2014.0,[{'index': 2247613}],"[{'name': '6531', 'w': 0.532050133}, {'name': ...",[1972366573]
...,...,...,...,...,...,...
4074424,2026775511,Bridging boolean and quantitative synthesis us...,2014.0,"[{'index': 107666}, {'index': 875472}, {'index...","[{'name': '1797', 'w': 0.5759903}, {'name': '1...","[88990729, 1490985464, 1510918931, 1527924265,..."
4074425,2026775633,Arrhythmia detection from heartbeat using k-ne...,2013.0,"[{'index': 341106}, {'index': 875473}, {'index...","[{'name': '132', 'w': 0.393144459}, {'name': '...","[1986307910, 1988183757, 1988566213, 199689436..."
4074429,2026775901,Organizational strategy development in distrib...,2012.0,"[{'index': 376798}, {'index': 875477}, {'index...","[{'name': '2112', 'w': 0.4589141}, {'name': '2...","[1967519272, 1972794857, 1997772982, 200124498..."
4074431,2026776168,Consideration effect of uncertainty in power s...,2011.0,"[{'index': 875478}, {'index': 875479}]","[{'name': '1429', 'w': 0.492216617}, {'name': ...","[1969487128, 2000969056, 2008743043, 204852993..."


In [60]:
len(filtered_years)

1777240

In [10]:
def get_authors_ids(references_list):
    authors = []
    if not isinstance(references_list, list) and pandas.isna(references_list):
        return []
    for reference_id in references_list:
        reference_authors = list(df.loc[df["id"] == int(reference_id)]["authors"])
        if len(reference_authors):
            authors.extend(list(map(lambda x: x["index"], reference_authors[0])))
    return list(set(authors))

In [37]:
classes = pandas.read_csv("classes_fos.csv", index_col=0)
agent_systems_class = classes.loc[classes["modularity_class"] == 20]

In [38]:
agent_systems_class["Id"].values

array([1048,  499, 1287, 1756, 1180])

In [70]:
def common_member(list_a, list_b):
    a_set = set(list_a)
    b_set = set(list_b)
    return len(a_set & b_set) > 0

In [66]:
filtered_with_wages = filtered_years.loc[filtered_years["fos"].apply(lambda x: type(x) is not float)]
filtered_with_wages

,id,title,year,authors,fos,references
5,1965664720,A low power wireless sensor node with vibratio...,2014.0,"[{'index': 2247601}, {'index': 1264008}, {'ind...","[{'name': '104', 'w': 0.0}, {'name': '649', 'w...","[1510600213, 2170239483]"
6,1965664774,Shortest Paths Based Web Service Selection in ...,2014.0,"[{'index': 2247603}, {'index': 2247604}]","[{'name': '2716', 'w': 0.5306294}, {'name': '2...","[1479916289, 1560267311, 2002838096, 200757299..."
12,1965665477,Contracts for First-Class Classes,2013.0,"[{'index': 465043}, {'index': 457705}, {'index...","[{'name': '10', 'w': 0.4498527}, {'name': '227...","[204817905, 1518650316, 1526601993, 1601712140..."
16,1965665804,An interpretation of the Cover and Leung capac...,2012.0,"[{'index': 283325}, {'index': 580777}]","[{'name': '0', 'w': 0.43447753800000005}, {'na...","[1489262227, 2011914761, 2014996477, 201797019..."
20,1965666279,Implementing Linguistic Landscape investigatio...,2014.0,[{'index': 2247613}],"[{'name': '6531', 'w': 0.532050133}, {'name': ...",[1972366573]
...,...,...,...,...,...,...
4074424,2026775511,Bridging boolean and quantitative synthesis us...,2014.0,"[{'index': 107666}, {'index': 875472}, {'index...","[{'name': '1797', 'w': 0.5759903}, {'name': '1...","[88990729, 1490985464, 1510918931, 1527924265,..."
4074425,2026775633,Arrhythmia detection from heartbeat using k-ne...,2013.0,"[{'index': 341106}, {'index': 875473}, {'index...","[{'name': '132', 'w': 0.393144459}, {'name': '...","[1986307910, 1988183757, 1988566213, 199689436..."
4074429,2026775901,Organizational strategy development in distrib...,2012.0,"[{'index': 376798}, {'index': 875477}, {'index...","[{'name': '2112', 'w': 0.4589141}, {'name': '2...","[1967519272, 1972794857, 1997772982, 200124498..."
4074431,2026776168,Consideration effect of uncertainty in power s...,2011.0,"[{'index': 875478}, {'index': 875479}]","[{'name': '1429', 'w': 0.492216617}, {'name': ...","[1969487128, 2000969056, 2008743043, 204852993..."


In [67]:
filtered_slice = filtered_years

In [68]:
filtered_slice["fos"] = filtered_with_wages["fos"].apply(lambda row: [int(field["name"]) for field in row])

In [71]:
filtered_slice = filtered_slice.loc[filtered_slice["fos"].apply(lambda foses: common_member(foses, agent_systems_class["Id"].values))]

In [72]:
len(filtered_slice)

11965

In [73]:
filtered_slice["author_references"] = filtered_slice["references"].apply(get_authors_ids)

/Users/bartek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
filtered_slice

,id,title,year,authors,fos,references,author_references
87,1965672995,Resource awareness in computational intelligence,2011.0,"[{'index': 35861}, {'index': 155234}, {'index'...","[1455, 132, 104, 672, 355, 4828, 4, 1048, 207,...","[80917968, 135902891, 179774170, 206495011, 15...","[550796, 2480915, 240788, 42390, 3583768, 2649..."
447,1965709183,Second-order group consensus for multi-agent s...,2015.0,"[{'index': 2247908}, {'index': 2247908}, {'ind...","[165, 6972, 42, 233, 499, 932, 935, 47]","[1966811233, 1974954027, 1985131825, 198543133...","[1666310, 2496519, 193670, 193671, 126346, 224..."
483,1965712808,An extensible toolbox for modeling nature-soci...,2013.0,"[{'index': 492477}, {'index': 1467334}, {'inde...","[10693, 441, 2249, 819, 1397, 579, 4, 10, 499,...","[1544752055, 1563849906, 1963843280, 198140440...","[937857, 116871, 1808903, 255240, 172176, 1167..."
794,1965745702,A novel agent-based concept of household appli...,2011.0,"[{'index': 2248185}, {'index': 111292}]","[506, 355, 4789, 0, 293, 499, 11240, 12992, 554]","[198400421, 1506992748, 1513230940, 1556082038...","[233890, 106884, 372325, 106886, 171242, 63211..."
1855,1965850651,Cluster-based instance selection for machine c...,2012.0,[{'index': 48145}],"[1940, 434, 1973, 3024, 132, 273, 1941, 1943, ...","[14890737, 165439655, 198984407, 1535979681, 1...","[335616, 262787, 1448580, 1855621, 1737226, 61..."
...,...,...,...,...,...,...,...
4073401,2026676438,A Family of Heuristics for Agent-Based Cloud B...,2011.0,"[{'index': 304132}, {'index': 319862}]","[0, 12458, 714, 590, 499, 4, 1287, 4676, 858, ...","[1992734168, 1996689181, 2031612682, 209702556...","[324480, 2944384, 266755, 304132, 474755, 2704..."
4073433,2026678777,Interleaving multi-agent systems and social ne...,2011.0,"[{'index': 89112}, {'index': 874269}, {'index'...","[53, 1227, 373, 127, 4, 1152, 5186, 499, 153, ...","[236418988, 1490468685, 1504739390, 1511711801...","[3607562, 281099, 943118, 191504, 790035, 8911..."
4073662,2026701147,Quantised consensus of multi-agent systems wit...,2015.0,"[{'index': 296652}, {'index': 296651}, {'index...","[42, 1088, 233, 12669, 396, 47, 499, 935]","[1982947074, 1991360560, 1996032233, 199910691...","[454273, 2496519, 50568, 44039, 1104010, 37004..."
4073703,2026705559,Enabling agent reasoning over the web,2013.0,"[{'index': 233187}, {'index': 233188}, {'index...","[127, 273, 9873, 1498, 172, 470, 11357, 32, 10...","[141753744, 1489021242, 1571255423, 1848570230...","[494855, 550796, 42390, 224286, 57758, 793516,..."


In [75]:
from networkx.algorithms.community.kclique import k_clique_communities

In [133]:
citation_dict = {}
def fill_graph(row, graph):
    authors = list(row["authors"])
    references_authors = list(row["author_references"])
    for author in list(authors):
        for refrence_author in references_authors:
            edge_name = str(author["index"]) + "_" + str(refrence_author)
            weight = graph.get(edge_name, 0)
            graph[edge_name] = weight + 1
            
filtered_slice.apply(lambda row: fill_graph(row, citation_dict), axis=1)

87         None
447        None
483        None
794        None
1855       None
           ... 
4073401    None
4073433    None
4073662    None
4073703    None
4073892    None
Length: 11965, dtype: object

In [151]:
import networkx
citation_graph = networkx.DiGraph()
for edge in citation_dict.keys():
    nodes = edge.split("_")
    node_x = int(nodes[0])
    node_y = int(nodes[1])
    weight = int(citation_dict[edge])
    citation_graph.add_edge(node_x, node_y, weight = weight)
citation_graph

In [152]:
citation_graph.number_of_edges()

1045412

In [154]:
networkx.write_gexf(citation_graph,"citation_graph_for_agent_systems.gexf")

In [210]:
def are_bidirectional_neighbors(a, b, g):
    return a in g.neighbors(b)

In [211]:
def get_bidirectional_nodes(graph):
    bidirectional_nodes = {}
    for element, adjacency in graph.adjacency():
        b_neighbors = list(map(lambda x: x[0], list(adjacency.items())))
        for neighbor in b_neighbors:
            if are_bidirectional_neighbors(element, neighbor, graph):
                element_bidirectional_neighbours = bidirectional_nodes.get(element, [])
                element_bidirectional_neighbours.append(neighbor)
                bidirectional_nodes[element] = element_bidirectional_neighbours
    return bidirectional_nodes
bidirectional_nodes = get_bidirectional_nodes(citation_graph)

In [221]:
sorted_bidirectional_nodes = [(k,v) for k, v in sorted(bidirectional_nodes.items(), key=lambda item: len(item[1]), reverse=True)]

In [249]:
len(sorted_bidirectional_nodes)

9281

In [244]:
top_bidirectional_nodes = []
for element, bidirectional_neighbors in sorted_bidirectional_nodes[:20]:
    top_bidirectional_nodes.append((element, len(bidirectional_neighbors)))

In [230]:
def get_author_name(row, graph):
    authors = list(row["authors"])
    references_authors = list(row["author_references"])
    for author in list(authors):
        for refrence_author in references_authors:
            edge_name = str(author["index"]) + "_" + str(refrence_author)
            weight = graph.get(edge_name, 0)
            graph[edge_name] = weight + 1
            
df["authors"].apply(lambda authors: fill_graph(row, citation_dict), axis=1)

[{'index': 2247598}, {'index': 190104}]

In [232]:
import json
authors_index_to_id = {}
with open('authors_index_to_id.json') as json_file:
    authors_index_to_id = json.load(json_file)

In [233]:
authors_id_name_map = {}
with open('authors_id__name_map.json') as json_file:
    authors_id_name_map = json.load(json_file)


In [248]:
for element, count in top_bidirectional_nodes:
    name = authors_id_name_map[authors_index_to_id[str(element)]]["name"]
    print("- %s - %d" % (name, count))

- Wei Ren - 296
- Guanrong Chen - 262
- Wenwu Yu - 233
- Long Wang - 208
- Jinde Cao - 193
- Nicholas R. Jennings - 192
- Lihua Xie - 149
- Frank L. Lewis - 146
- Guanghui Wen - 144
- Gang Feng - 136
- Michael Luck - 134
- Yongcan Cao - 129
- Dimos V. Dimarogonas - 125
- Wei Xing Zheng - 123
- Yiguang Hong - 122
- Huijun Gao - 121
- Zidong Wang - 113
- Housheng Su - 110
- Zhisheng Duan - 110
- Karl Henrik Johansson - 110
